In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
from pandas.tseries.offsets import Hour
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from tensorflow.keras.utils import set_random_seed as set_seed
set_seed(42)


In [2]:


#df = pd.read_csv('Wind_deseason.csv')
df = pd.read_csv('PV_deseason_15.csv', parse_dates=[0], index_col=0)


In [3]:
def TrainTest(df):
    
    num_rows = df.shape[0]
    
    num_training = (num_rows * 8)//10
   
    #train = df.iloc[0:num_training]
    #test = df.iloc[num_training:num_rows]
    
    
    return num_training 
        
            

In [4]:
def lags(df, n):
    
    df_new = df
    
    for i in range(n):
        name = "lag "+ str(i+1)
        df_new[name] = df['310_PV_1'].shift(i+1)
    
    return df_new

In [5]:

def model1Train(X,Y):
     
    mod = Sequential()
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1), return_sequences = True))
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1), return_sequences = True))
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1)))
    mod.add(Dense(1))
    mod.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
       
        
    print(mod.summary())
        
    mod.fit(x = X, y = Y,epochs=20,verbose = 1)#keep low for now, edit later
    
    return mod
    
    
    

In [6]:
def model2Train(gen):
     
    mod = Sequential()
    mod.add(LSTM(1, activation='sigmoid', input_shape=(107, 1)))
   
    mod.add(Dense(1))
    mod.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
       
        
    print(mod.summary())
        
    mod.fit(gen,epochs=5,verbose = 1)#keep low for now, edit later
    
    return mod
    

In [7]:
def TestModel(X,test, mod, scaler):
    
    MSEs = []
    
    for i in range(len(X)):
        array = np.array(X[i])
        
        pred = mod.predict(array.reshape(1,107,1))
        print(pred)
        #pred = scaler.inverse_transform(pred)
        unscaled_pred = scaler.inverse_transform(pred)
        unscaled_test = scaler.inverse_transform([[test[i]]])
       
        
        MSEs.append(mse(unscaled_test[0],unscaled_pred[0]))
    return MSEs

In [8]:
#def modelNaiveTest(df):

    
    
    


In [9]:
def runModels(df, last):
    
    scaler = MinMaxScaler()
    
    bound = TrainTest(df)
    
    train = df[0:bound]
    test = df[bound:]
    
    
    scaler.fit(train['310_PV_1'].to_frame())
    
    scaled_df = scaler.transform(df['310_PV_1'].to_frame())
    scaled_last = scaler.transform(last['310_PV_1'].to_frame())
    scaled_df = np.append(scaled_last, scaled_df)
    
    generator1 =  TimeseriesGenerator(scaled_df, scaled_df, length=107, batch_size=1)
    generator = TimeseriesGenerator(scaled_df[107:bound+107], scaled_df[107:bound+107], length=107, batch_size=1)
    
    
    scaled_df2 = pd.DataFrame(scaled_df, columns = ['310_PV_1'])
    
    lag = lags(scaled_df2, 107)
    
    lag = lag[lag.columns[::-1]]
    
    lagged_train = lag.iloc[107:bound+107]
    lagged_test = lag.iloc[bound+107:]

    
    
    X_test = lagged_test.drop('310_PV_1', axis = 1).values.tolist()
    Y_test = lagged_test['310_PV_1'].values.tolist()
  
    
    model2 = model2Train(generator)
    
    Tests = TestModel(X_test, Y_test, model2, scaler)
    
    
    print(Tests)
    
   
    
    return Tests#, mse2

    

In [10]:


df1 = df[df.index.month.isin([1,2,3])]
df1 = df1.iloc[:-1 , :]


df2 = df[df.index.month.isin([4,5,6])]

df3 = df[df.index.month.isin([7,8,9])]

df4 = df[df.index.month.isin([10,11,12])]

In [11]:
 #mse1 = runModels(df1,df4.iloc[-107:])


In [12]:
#mse3 = runModels(df2,df1.iloc[-107:])

In [13]:
#mse5 = runModels(df3, df2.iloc[-107:])

In [14]:
mse7 = runModels(df4, df3.iloc[-107:])

C:\Users\bvits\AppData\Local\Temp\ipykernel_15888\610209718.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[name] = df['310_PV_1'].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_15888\610209718.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[name] = df['310_PV_1'].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_15888\610209718.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1)                 12        
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
6958/6958 [==============================] - 176s 25ms/step - loss: 0.0066 - accuracy: 2.8744e-04
Epoch 2/5
6958/6958 [==============================] - 176s 25ms/step - loss: 0.0039 - accuracy: 2.8744e-04
Epoch 3/5
6958/6958 [==============================] - 177s 25ms/step - loss: 0.0031 - accuracy: 2.8744e-04
Epoch 4/5
6958/6958 [==============================] - 184s 26ms/step - loss: 0.0028 - accuracy: 2.8744e-04
Epoch 5/5
1/1 [====

1/1 [==============================] - 0s 21ms/step
[[0.49394125]]
1/1 [==============================] - 0s 23ms/step
[[0.37336904]]
1/1 [==============================] - 0s 21ms/step
[[0.449544]]
1/1 [==============================] - 0s 22ms/step
[[0.51825917]]
1/1 [==============================] - 0s 23ms/step
[[0.5726483]]
1/1 [==============================] - 0s 23ms/step
[[0.6264513]]
1/1 [==============================] - 0s 23ms/step
[[0.6620241]]
1/1 [==============================] - 0s 20ms/step
[[0.7048957]]
1/1 [==============================] - 0s 22ms/step
[[0.7481894]]
1/1 [==============================] - 0s 22ms/step
[[0.7796772]]
1/1 [==============================] - 0s 23ms/step
[[0.7900378]]
1/1 [==============================] - 0s 21ms/step
[[0.7927436]]
1/1 [==============================] - 0s 41ms/step
[[0.79332894]]
1/1 [==============================] - 0s 22ms/step
[[0.7939121]]
1/1 [==============================] - 0s 42ms/step
[[0.7943761]]
1/1 [==

1/1 [==============================] - 0s 23ms/step
[[0.797399]]
1/1 [==============================] - 0s 23ms/step
[[0.79746765]]
1/1 [==============================] - 0s 23ms/step
[[0.7975289]]
1/1 [==============================] - 0s 24ms/step
[[0.7975835]]
1/1 [==============================] - 0s 23ms/step
[[0.7976323]]
1/1 [==============================] - 0s 20ms/step
[[0.7976764]]
1/1 [==============================] - 0s 20ms/step
[[0.7977161]]
1/1 [==============================] - 0s 21ms/step
[[0.79775196]]
1/1 [==============================] - 0s 21ms/step
[[0.7977846]]
1/1 [==============================] - 0s 20ms/step
[[0.7978144]]
1/1 [==============================] - 0s 21ms/step
[[0.79784185]]
1/1 [==============================] - 0s 22ms/step
[[0.7978669]]
1/1 [==============================] - 0s 22ms/step
[[0.7978901]]
1/1 [==============================] - 0s 21ms/step
[[0.7979116]]
1/1 [==============================] - 0s 21ms/step
[[0.79793173]]
1/1 [==

[[0.7514717]]
1/1 [==============================] - 0s 21ms/step
[[0.7103581]]
1/1 [==============================] - 0s 20ms/step
[[0.6511732]]
1/1 [==============================] - 0s 21ms/step
[[0.6101803]]
1/1 [==============================] - 0s 20ms/step
[[0.5607451]]
1/1 [==============================] - 0s 22ms/step
[[0.49217814]]
1/1 [==============================] - 0s 20ms/step
[[0.41565257]]
1/1 [==============================] - 0s 44ms/step
[[0.4023986]]
1/1 [==============================] - 0s 24ms/step
[[0.55558836]]
1/1 [==============================] - 0s 20ms/step
[[0.64875096]]
1/1 [==============================] - 0s 21ms/step
[[0.7050702]]
1/1 [==============================] - 0s 22ms/step
[[0.7172267]]
1/1 [==============================] - 0s 22ms/step
[[0.72810096]]
1/1 [==============================] - 0s 21ms/step
[[0.7535096]]
1/1 [==============================] - 0s 22ms/step
[[0.7676199]]
1/1 [==============================] - 0s 21ms/step
[[0.7

...

1/1 [==============================] - 0s 27ms/step
[[0.80153686]]
1/1 [==============================] - 0s 40ms/step
[[0.7780804]]
1/1 [==============================] - 0s 22ms/step
[[0.7593296]]
1/1 [==============================] - 0s 22ms/step
[[0.7183761]]
1/1 [==============================] - 0s 20ms/step
[[0.65935373]]
1/1 [==============================] - 0s 22ms/step
[[0.61843854]]
1/1 [==============================] - 0s 22ms/step
[[0.56906766]]
1/1 [==============================] - 0s 24ms/step
[[0.50050557]]
1/1 [==============================] - 0s 21ms/step
[[0.42388833]]
1/1 [==============================] - 0s 23ms/step
[[0.3626927]]
1/1 [==============================] - 0s 22ms/step
[[0.5228752]]
1/1 [==============================] - 0s 26ms/step
[[0.6447379]]
1/1 [==============================] - 0s 24ms/step
[[0.7231764]]
1/1 [==============================] - 0s 24ms/step
[[0.74831635]]
1/1 [==============================] - 0s 23ms/step
[[0.7790292]]
1/1

1/1 [==============================] - 0s 23ms/step
[[0.740552]]
1/1 [==============================] - 0s 22ms/step
[[0.732652]]
1/1 [==============================] - 0s 22ms/step
[[0.73555285]]
1/1 [==============================] - 0s 22ms/step
[[0.729369]]
1/1 [==============================] - 0s 22ms/step
[[0.72074443]]
1/1 [==============================] - 0s 21ms/step
[[0.7388206]]
1/1 [==============================] - 0s 21ms/step
[[0.71404225]]
1/1 [==============================] - 0s 21ms/step
[[0.70129436]]
1/1 [==============================] - 0s 20ms/step
[[0.70024544]]
1/1 [==============================] - 0s 19ms/step
[[0.69853455]]
1/1 [==============================] - 0s 20ms/step
[[0.7037919]]
1/1 [==============================] - 0s 35ms/step
[[0.70635766]]
1/1 [==============================] - 0s 22ms/step
[[0.70976406]]
1/1 [==============================] - 0s 23ms/step
[[0.6984437]]
1/1 [==============================] - 0s 19ms/step
[[0.6771461]]
1/1 [

1/1 [==============================] - 0s 21ms/step
[[0.80511624]]
1/1 [==============================] - 0s 20ms/step
[[0.80573934]]
1/1 [==============================] - 0s 20ms/step
[[0.80621094]]
1/1 [==============================] - 0s 48ms/step
[[0.806595]]
1/1 [==============================] - 0s 47ms/step
[[0.8069156]]
1/1 [==============================] - 0s 27ms/step
[[0.8071844]]
1/1 [==============================] - 0s 21ms/step
[[0.80741197]]
1/1 [==============================] - 0s 22ms/step
[[0.80760616]]
1/1 [==============================] - 0s 23ms/step
[[0.80777246]]
1/1 [==============================] - 0s 21ms/step
[[0.8079161]]
1/1 [==============================] - 0s 22ms/step
[[0.80804056]]
1/1 [==============================] - 0s 23ms/step
[[0.80814904]]
1/1 [==============================] - 0s 23ms/step
[[0.80824417]]
1/1 [==============================] - 0s 21ms/step
[[0.8083276]]
1/1 [==============================] - 0s 20ms/step
[[0.8084015]]
1/

1/1 [==============================] - 0s 24ms/step
[[0.80971783]]
1/1 [==============================] - 0s 20ms/step
[[0.8097449]]
1/1 [==============================] - 0s 22ms/step
[[0.8097698]]
1/1 [==============================] - 0s 20ms/step
[[0.80979294]]
1/1 [==============================] - 0s 23ms/step
[[0.8098143]]
1/1 [==============================] - 0s 21ms/step
[[0.8098343]]
1/1 [==============================] - 0s 22ms/step
[[0.8098528]]
1/1 [==============================] - 0s 21ms/step
[[0.8098704]]
1/1 [==============================] - 0s 20ms/step
[[0.809887]]
1/1 [==============================] - 0s 22ms/step
[[0.8099026]]
1/1 [==============================] - 0s 20ms/step
[[0.8099174]]
1/1 [==============================] - 0s 47ms/step
[[0.8099317]]
1/1 [==============================] - 0s 33ms/step
[[0.80994517]]
1/1 [==============================] - 0s 21ms/step
[[0.80995816]]
1/1 [==============================] - 0s 21ms/step
[[0.80997056]]
1/1 [=

1/1 [==============================] - 0s 22ms/step
[[0.7635582]]
1/1 [==============================] - 0s 21ms/step
[[0.77988905]]
1/1 [==============================] - 0s 22ms/step
[[0.80221254]]
1/1 [==============================] - 0s 21ms/step
[[0.82212895]]
1/1 [==============================] - 0s 24ms/step
[[0.8281953]]
1/1 [==============================] - 0s 21ms/step
[[0.82917994]]
1/1 [==============================] - 0s 20ms/step
[[0.8061437]]
1/1 [==============================] - 0s 24ms/step
[[0.79596204]]
1/1 [==============================] - 0s 20ms/step
[[0.80626315]]
1/1 [==============================] - 0s 22ms/step
[[0.79819936]]
1/1 [==============================] - 0s 22ms/step
[[0.8035006]]
1/1 [==============================] - 0s 23ms/step
[[0.7992396]]
1/1 [==============================] - 0s 22ms/step
[[0.7867184]]
1/1 [==============================] - 0s 21ms/step
[[0.78635913]]
1/1 [==============================] - 0s 21ms/step
[[0.7671197]]
1/

In [15]:
#print('season 1')

#print('model 1:')
#print(mse1)
#print('model Naive:')
#print(mse2)


In [16]:
#print('season 1')

#print('model 1:')
#print(mse3)
#print('model Naive:')
#print(mse4)


In [17]:
#print('season 1')

#print('model 1:')
#print(mse5)
#print('model Naive:')
#print(mse6)


In [18]:
#print('season 1')

#print('model 1:')
#print(mse7)
#print('model Naive:')
#print(mse8)


In [19]:

#d = {'Season4':mse7}
#df_errors_LSTM = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})
#df_errors_LSTM.to_csv('LSTM_errors_scaled_season4.csv')
#'Season1':mse1,'Season2':mse3,'Season3':mse5,
